In [21]:
import pandas as pd
import string

#Payloads
xss = pd.read_csv("/content/drive/MyDrive/data/xss.csv")
sqli = pd.read_csv("/content/drive/MyDrive/data/sqli.csv")
cmdi = pd.read_csv("/content/drive/MyDrive/data/cmdi.csv")
legit = pd.read_csv("/content/drive/MyDrive/data/legit.csv")

# Wordlists
jsKeys = pd.read_csv("/content/drive/MyDrive/data/jsKeys.csv")
sqlKeys = pd.read_csv("/content/drive/MyDrive/data/sqlKeys.csv")
cmdKeys = pd.read_csv("/content/drive/MyDrive/data/cmdKeys.csv")

In [22]:
# Combining all data
temp = pd.concat([xss, sqli, cmdi, legit], ignore_index=True)
data = temp.drop_duplicates()
print(len(data))

90638


In [23]:
# Feature 1: Length
def findLength(s):
  return len(str(s))

# Feature 2: Number of non-printable characters
def findNonPrintChars(s):
  count = 0
  for char in str(s):
    # Check if the character is a printable ASCII character or a whitespace character
    if ord(char) < 32 or ord(char) > 126 or char.isspace():
      count += 1
  return count

# Feature 3: Number of special characters
def findSpecialChars(s):
  count = 0
  for char in str(s):
    if char in string.punctuation:
      count += 1
  return count

def findSqlKeys(s):
  count = 0
  for item in sqlKeys['Keyword']:
    if str(item).lower() in str(s).lower():
      count += 1
  return count

def findJsKeys(s):
  count = 0
  for item in jsKeys['Keyword']:
    if str(item).lower() in str(s).lower():
      count += 1
  return count

def findCmdKeys(s):
  count = 0
  for item in cmdKeys['Keyword']:
    if str(item).lower() in str(s).lower():
      count += 1
  return count

In [24]:
# Feature Extraction
def extractFeatures(payload, label):
  features = []
  #features.append(payload)
  features.append(findLength(payload))
  features.append(findNonPrintChars(payload))
  features.append(findSpecialChars(payload))
  features.append(findSqlKeys(payload))
  features.append(findJsKeys(payload))
  features.append(findCmdKeys(payload))
  #features.append(label)
  features.append(label)
  return features

In [25]:
features = []
errors = 0

for i in range(0,len(data)):
  try:
    payload = data['payload'][i]
    label = data['attack'][i]

    features.append(extractFeatures(payload, label))

  except:
    errors += 1
    pass

print(len(features))
print(errors)

88789
1849


In [26]:
feature_names = ['length', 'nonPrint', 'specialChars', 'sqlKeys', 'jsKeys', 'cmdKeys', 'label']

allFeatures = pd.DataFrame(features, columns= feature_names)

allFeatures.head()

,length,nonPrint,specialChars,sqlKeys,jsKeys,cmdKeys,label
0,13,0,6,0,1,1,XSS
1,12,0,5,0,1,1,XSS
2,16,0,8,0,1,1,XSS
3,15,0,7,0,1,1,XSS
4,35,0,16,1,3,0,XSS


In [27]:
# Saving Extracted Features in File
allFeatures.to_csv('allFeatures.csv', index= False)

In [28]:
testingData = allFeatures.sample(100)
testingData.to_csv('testingData.csv', index= False)